<a href="https://colab.research.google.com/github/kchamarty/pythonproject/blob/patienctchurn/kumarchamarty_pythonv1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Data Importing**


---

We have two data sets one containing the patient doctor visit information and the other containing the status of the patient active or not

In [0]:
#importing packages

import pandas as pd # Load the pandas libraries with alias 'pd'
 
import seaborn as sns # Load the seaborn libraries with alias 'sns'

import matplotlib.pyplot as plt # Load the matplotlib.pyplot libraries with alias 'plt' 

from pandas_profiling import ProfileReport as pr

In [0]:
patient_doc_visit_df = pd.read_csv("https://raw.githubusercontent.com/kchamarty/pythonproject/master/data/PatientDoctorVisit.csv");

patient_churn_df = pd.read_csv(r"https://raw.githubusercontent.com/kchamarty/pythonproject/master/data/PatientDetails.csv");

patient_train_df = pd.read_csv(r"https://raw.githubusercontent.com/kchamarty/pythonproject/master/data/Train_Patient_Churn.csv");

patient_test_df = pd.read_csv(r"https://raw.githubusercontent.com/kchamarty/pythonproject/master/data/Test_Patient_Churn.csv");


In [0]:
import pandas_profiling


In [12]:
pr(patient_doc_visit_df)

/usr/local/lib/python3.6/dist-packages/pandas_profiling/describe.py:392: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.
  variable_stats = pd.concat(ldesc, join_axes=pd.Index([names]), axis=1)


Number of variables,8
Number of observations,573401
Total Missing (%),0.3%
Total size in memory,35.0 MiB
Average record size in memory,64.0 B
Numeric,2
Categorical,6
Boolean,0
Date,0
Text (Unique),0
Rejected,0


In [14]:
from sklearn.utils import resample


RangeIndex(start=0, stop=573401, step=1)

In [17]:
x= resample(patient_doc_visit_df.index, replace=True, n_samples=4, random_state=1)
patient_doc_visit_df.loc[x,:]

,transaction_date,practice_id,patient_id,provider_id,transaction_type,proceedure_code,proceedure_description,amount
128037,11/20/2017,D8066,51996,H5,Procedure,11113,Scaling Three Units of Time,123.0
491755,6/24/2019,D8065,2890,HYG4,Procedure,11117,Scaling 1/2 unit,22.0
470924,6/22/2018,D8066,56039,H3,Procedure,11113,Scaling Three Units of Time,126.0
491263,7/9/2019,D8065,967,HYGA,Procedure,11107,Polish 1/2 unit,25.0


# BootStrap function

In [24]:
#Definition of Bootstrap
#Choose a number of bootstrap samples to perform
#Choose a sample size
#For each bootstrap sample
#Draw a sample with replacement with the chosen size
#Calculate the statistic on the sample
#Calculate the mean of the calculated sample statistics.

def boot_samp_repl(df,sample_size, numberofsamples):
  bootsamples=dict();
  for n in range(1,numberofsamples+1):
    i = resample(df.index, replace=True, n_samples=sample_size, random_state=1)
    bootsamples["sample_{}".format(n)]=df.loc[i,:]
  return bootsamples

x=boot_samp_repl(patient_doc_visit_df,4,10)
x




{'sample_1':        transaction_date practice_id  ...       proceedure_description amount
 128037       11/20/2017       D8066  ...  Scaling Three Units of Time  123.0
 491755        6/24/2019       D8065  ...             Scaling 1/2 unit   22.0
 470924        6/22/2018       D8066  ...  Scaling Three Units of Time  126.0
 491263         7/9/2019       D8065  ...              Polish 1/2 unit   25.0
 
 [4 rows x 8 columns],
 'sample_2':        transaction_date practice_id  ...       proceedure_description amount
 128037       11/20/2017       D8066  ...  Scaling Three Units of Time  123.0
 491755        6/24/2019       D8065  ...             Scaling 1/2 unit   22.0
 470924        6/22/2018       D8066  ...  Scaling Three Units of Time  126.0
 491263         7/9/2019       D8065  ...              Polish 1/2 unit   25.0
 
 [4 rows x 8 columns],
 'sample_3':        transaction_date practice_id  ...       proceedure_description amount
 128037       11/20/2017       D8066  ...  Scaling Three

In [23]:
bootsamples

{'sample_1':        transaction_date practice_id  ...       proceedure_description amount
 128037       11/20/2017       D8066  ...  Scaling Three Units of Time  123.0
 491755        6/24/2019       D8065  ...             Scaling 1/2 unit   22.0
 470924        6/22/2018       D8066  ...  Scaling Three Units of Time  126.0
 491263         7/9/2019       D8065  ...              Polish 1/2 unit   25.0
 
 [4 rows x 8 columns],
 'sample_2':        transaction_date practice_id  ...       proceedure_description amount
 128037       11/20/2017       D8066  ...  Scaling Three Units of Time  123.0
 491755        6/24/2019       D8065  ...             Scaling 1/2 unit   22.0
 470924        6/22/2018       D8066  ...  Scaling Three Units of Time  126.0
 491263         7/9/2019       D8065  ...              Polish 1/2 unit   25.0
 
 [4 rows x 8 columns],
 'sample_3':        transaction_date practice_id  ...       proceedure_description amount
 128037       11/20/2017       D8066  ...  Scaling Three